# Jupyter/Colab Notebook to Showcase sparkMeasure APIs for Python

### [Run on Google Colab Research: <img src="https://raw.githubusercontent.com/googlecolab/open_in_colab/master/images/icon128.png">](https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb)

**SparkMeasure is a tool for performance troubleshooting of Apache Spark workloads**  
It simplifies the collection and analysis of Spark performance metrics. It is also intended as a working example of how to use Spark listeners for collecting and processing Spark executors task metrics data.

**References:**
- [https://github.com/LucaCanali/sparkMeasure](https://github.com/LucaCanali/sparkMeasure)  
- sparkmeasure Python docs: [docs/Python_shell_and_Jupyter](https://github.com/LucaCanali/sparkMeasure/blob/master/docs/Python_shell_and_Jupyter.md)  

**Architecture:**
![sparkMeasure architecture diagram](https://github.com/LucaCanali/sparkMeasure/raw/master/docs/sparkMeasure_architecture_diagram.png)

Author and contact: Luca.Canali@cern.ch  
Last updated: March 2024

In [1]:
# Dependencies

# Install PySpark/Spark 

!pip install pyspark

# Install the Python wrapper API for spark-measure

!pip install sparkmeasure

In [ ]:
from pyspark.sql import SparkSession

# Start the Spark Session
# This example uses Spark in local mode for simplicity.
# You can modify master to use  YARN or K8S if available 
# This example uses sparkMeasure 0.24 for scala 2.12, taken from maven central

spark = SparkSession \
 .builder \
 .master("local[*]") \
 .appName("Test sparkmeasure instrumentation of Python/PySpark code") \
 .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.12:0.25")  \
 .getOrCreate()

In [2]:
# Test that Spark is working OK
spark.sql("select 1 as id, 'Hello world!' as Greeting").show()

+---+------------+
| id|    Greeting|
+---+------------+
|  1|Hello world!|
+---+------------+



In [3]:
# Initialize sparkMeasure
# Load the Python API for sparkmeasure package
# and attach the sparkMeasure Listener for stagemetrics to the active Spark session

from sparkmeasure import StageMetrics
stagemetrics = StageMetrics(spark)

In [4]:
# The easiest way to start using sparkMesure is with the "runandmeasure" method
# This will execute your Spark action, return the results, and collect and aggregate execution metrics

stagemetrics.runandmeasure(globals(), """
spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()
""")

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 8

Aggregated Spark stage metrics:
numStages => 3
numTasks => 17
elapsedTime => 718 (0.7 s)
stageDuration => 473 (0.5 s)
executorRunTime => 2518 (3 s)
executorCpuTime => 2200 (2 s)
executorDeserializeTime => 239 (0.2 s)
executorDeserializeCpuTime => 102 (0.1 s)
resultSerializationTime => 21 (21 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 34 (34 ms)
resultSize => 16306 (15.9 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 8
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 472 (472 Bytes)
shuffleLocalBytesRead => 472 (472 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDi

In [6]:
# Additionally, sparkMeasure collects executor metrics
# This is how you can print the memory usage report
# Note, If you receive the error message java.util.NoSuchElementException: key not found, 
# retry running the report after waiting for a few seconds.

stagemetrics.print_memory_report()


Additional stage-level executor metrics (memory usage info):

Stage 1 JVMHeapMemory maxVal bytes => 204990016 (195.5 MB)
Stage 1 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 2 JVMHeapMemory maxVal bytes => 204990016 (195.5 MB)
Stage 2 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)
Stage 4 JVMHeapMemory maxVal bytes => 204990016 (195.5 MB)
Stage 4 OnHeapExecutionMemory maxVal bytes => 0 (0 Bytes)


In [7]:
# An equivalent API for collecting execution metrics is to explicitly wrap your Spark workload
# into stagemetrics instrumentation, as in this example

stagemetrics.begin()

spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()

stagemetrics.end()

# Print a summary report
stagemetrics.print_report()

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 8

Aggregated Spark stage metrics:
numStages => 3
numTasks => 17
elapsedTime => 444 (0.4 s)
stageDuration => 361 (0.4 s)
executorRunTime => 2156 (2 s)
executorCpuTime => 1708 (2 s)
executorDeserializeTime => 87 (87 ms)
executorDeserializeCpuTime => 42 (42 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 8 (8 ms)
resultSize => 16048 (15.7 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 8
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 472 (472 Bytes)
shuffleLocalBytesRead => 472 (472 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 

In [8]:
# This code is for Jupyter notebooks
# Define cell and line magic to wrap sparkmeasure instrumentation
# See example in the next cell

from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

@register_line_cell_magic
def sparkmeasure(line, cell=None):
    "run and measure spark workload. Use: %sparkmeasure or %%sparkmeasure"
    val = cell if cell is not None else line
    stagemetrics.begin()
    eval(val)
    stagemetrics.end()
    stagemetrics.print_report()

In [9]:
%%sparkmeasure
spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 8

Aggregated Spark stage metrics:
numStages => 3
numTasks => 17
elapsedTime => 361 (0.4 s)
stageDuration => 297 (0.3 s)
executorRunTime => 1819 (2 s)
executorCpuTime => 1738 (2 s)
executorDeserializeTime => 51 (51 ms)
executorDeserializeCpuTime => 41 (41 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 8 (8 ms)
resultSize => 16048 (15.7 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 8
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 472 (472 Bytes)
shuffleLocalBytesRead => 472 (472 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 

## Example of using Task Metrics
Collecting Spark task metrics at the granularity of each task completion has additional overhead
compared to collecting at the stage-level.
This option should only be used if you need data with this finer granularity, for example because you want
to study skew effects, otherwise consider using stagemetrics aggregation as the preferred choice.


In [10]:
from sparkmeasure import TaskMetrics
taskmetrics = TaskMetrics(spark)

taskmetrics.begin()
spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()
taskmetrics.end()

taskmetrics.print_report()

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 8

Aggregated Spark task metrics:
numTasks => 17
successful tasks => 17
speculative tasks => 0
taskDuration => 2102 (2 s)
schedulerDelayTime => 84 (84 ms)
executorRunTime => 1955 (2 s)
executorCpuTime => 1715 (2 s)
executorDeserializeTime => 63 (63 ms)
executorDeserializeCpuTime => 33 (33 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 1 (1 ms)
gettingResultTime => 0 (0 ms)
resultSize => 2667 (2.6 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 8
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 472 (472 Bytes)
shuffleLocalBytesRead => 472 (472 Bytes)
shuffleR

In [11]:
taskmetrics.runandmeasure(globals(), """
spark.sql("select count(*) from range(1000) cross join range(1000) cross join range(1000)").show()
""")

+----------+
|  count(1)|
+----------+
|1000000000|
+----------+


Scheduling mode = FIFO
Spark Context default degree of parallelism = 8

Aggregated Spark task metrics:
numTasks => 17
successful tasks => 17
speculative tasks => 0
taskDuration => 2192 (2 s)
schedulerDelayTime => 81 (81 ms)
executorRunTime => 2045 (2 s)
executorCpuTime => 1780 (2 s)
executorDeserializeTime => 66 (66 ms)
executorDeserializeCpuTime => 30 (30 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
gettingResultTime => 0 (0 ms)
resultSize => 2667 (2.6 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 2000
bytesRead => 0 (0 Bytes)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 8
shuffleTotalBlocksFetched => 8
shuffleLocalBlocksFetched => 8
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 472 (472 Bytes)
shuffleLocalBytesRead => 472 (472 Bytes)
shuffleR

In [12]:
spark.stop()